In [5]:
import warnings
warnings.filterwarnings("ignore")
import os
os.chdir("D:")
import numpy as np
import pandas as pd
import jaqs_fxdayu
jaqs_fxdayu.patch_all()
from jaqs.data import DataView
from jaqs_fxdayu.data.dataservice import LocalDataService
from jaqs_fxdayu.data import signal_function_mod as sfm
from jaqs_fxdayu.research.signaldigger import process
import jaqs.util as jutill
import scipy.stats as scst

dataview_folder = '../data'
dv = DataView()
ds = LocalDataService(fp=dataview_folder)

In [6]:
#--------------------------------------------------------
#define

start = 20170101
end = 20180101
#factor_list  = ['BBI','RVI','Elder','ChaikinVolatility','EPS','PE','PS','ACCA','CTOP','MA10RegressCoeff12','AR','BR','ARBR','np_parent_comp_ttm','total_share','bps']
factor_list = ['turnover_ratio','volume','tot_non_cur_liab','tot_liab_shrhldr_eqy','less_selling_dist_exp','total_oper_rev','undistributed_profit','capital_stk']
check_factor = ','.join(factor_list)

# 非流动负债合计：tot_non_cur_liab
# 股东权益：tot_liab_shrhldr_eqy
# 销售费用：less_selling_dist_exp
# 营业总收入：total_oper_rev
# 未分配利润：undistributed_profit
# 股本：capital_stk

dataview_folder = r'D:/data'
ds = LocalDataService(fp = dataview_folder)

SH_id = ds.query_index_member("000001.SH", start, end)
SZ_id = ds.query_index_member("399106.SZ", start, end)
stock_symbol = list(set(SH_id)|set(SZ_id))

dv_props = {'start_date': start, 'end_date': end, 'symbol':','.join(stock_symbol),
         'fields': check_factor,
         'freq': 1,
         "prepare_fields": True}

dv = DataView()
dv.init_from_config(dv_props, data_api=ds)
dv.prepare_data()

Initialize config success.
Query data...
Query data - query...
NOTE: price adjust method is [post adjust]
当前请求daily...
{'adjust_mode': None, 'fields': 'high_adj,volume,symbol,close_adj,close,open,trade_date,high,trade_status,vwap_adj,low_adj,vwap,open_adj,low'}
下载进度1760/3512.
当前请求daily...
{'adjust_mode': 'post', 'fields': 'close,open,high,vwap,low,symbol,trade_date'}
下载进度1760/3512.
当前请求query_lb_dailyindicator...
{'fields': 'trade_date,turnover_ratio,symbol'}
下载进度1760/3512.
    At fields 000748.SZ, 200011.SZ, 200012.SZ, 200016.SZ, 200017.SZ, 200018.SZ, 200019.SZ, 200020.SZ, 200022.SZ, 200025.SZ, 200026.SZ, 200028.SZ, 200029.SZ, 200030.SZ, 200037.SZ, 200045.SZ, 200053.SZ, 200054.SZ, 200055.SZ, 200056.SZ, 200058.SZ, 200152.SZ, 200160.SZ, 200168.SZ, 200413.SZ, 200418.SZ, 200429.SZ, 200468.SZ, 200488.SZ, 200505.SZ, 200512.SZ, 200521.SZ, 200530.SZ, 200539.SZ, 200541.SZ, 200550.SZ, 200553.SZ, 200570.SZ, 200581.SZ, 200596.SZ, 200613.SZ, 200625.SZ, 200706.SZ, 200725.SZ, 200726.SZ, 200761.SZ, 20

In [7]:
# dv.fields

In [8]:
BLEV_ME = dv.add_formula('BLEV_ME',"tot_non_cur_liab/(tot_non_cur_liab+tot_liab_shrhldr_eqy)",is_quarterly=True,add_data=True)
OperatingExpenesRate = dv.add_formula('OperatingExpenesRate',"less_selling_dist_exp/total_oper_rev",is_quarterly=True,add_data=True)
UndivdedProfitPS = dv.add_formula('UndivdedProfitPS',"undistributed_profit/capital_stk",is_quarterly=True,add_data=True)
VOL121 = dv.add_formula("VOL121","Ts_Mean(turnover_ratio,{})".format(120),overwrite=True,is_quarterly=False,add_data=True)
VOL201 = dv.add_formula("VOL201","Ts_Mean(turnover_ratio,{})".format(20),overwrite=True,is_quarterly=False,add_data=True)
alpha137 = dv.add_formula("alpha137","16*(close-Delay(close,1)+(close-open)/2+Delay(close,1)-Delay(open,1))/If((Abs(high-Delay(close,1))>Abs(low-Delay(close,1))&&Abs(high-Delay(close,1))>Abs(high-Delay(low,1)),Abs(high-Delay(close,1))+Abs(low-Delay(close,1))/2+Abs(Delay(close,1)-Delay(open,1))/4,If(Abs(low-Delay(close,1))>Abs(high-Delay(low,1))&&Abs(low-Delay(close,1))>Abs(high-Delay(close,1)),Abs(low-Delay(close,1))+Abs(high-Delay(close,1))/2+Abs(Delay(close,1)-Delay(open,1))/4,Abs(high-Delay(low,1))+Abs(Delay(close,1)-Delay(open,1))/4)))*Max(Abs(high-Delay(close,1)),Abs(low-Delay(close,1)))",is_quarterly=False,add_data=True)


In [9]:
def spearman_factor(factor_u='factor_u', factor_j='factor_j', start=20160101, symbol=stock_symbol):
    factors = dict()
    factors["factor_u"] = dv.get_ts(factor_u).loc[start:][stock_symbol].stack()
    factors["factor_j"] = dv.get_ts(factor_j).loc[start:][stock_symbol].stack()

    factors_df = pd.concat(factors.values(),axis=1)
    factors_df.columns = factors.keys()
    factors_df = jutill.fillinf(factors_df).dropna()

    # 两个因子的spearman相关性
    def get_factors_daily_corr(factors_df,
                               f1="factor_u",
                               f2="factor_j"):
        def _cal_corr(df):
            return scst.spearmanr(df[f1], df[f2])[0]

        result = factors_df.groupby(['trade_date']).apply(_cal_corr)
        result = pd.DataFrame(result)
        result.columns = ['spearmanr']
        return result

    result = get_factors_daily_corr(factors_df,"factor_u","factor_j")
    t_stat, p_value = scst.ttest_1samp(result, 0)
    return result.mean(), p_value # 相关关系均值,相关关系均值的ｔ统计量的p_value

In [10]:
#BLEV因子复现检验
dv.add_field('BLEV', ds)
BLEV = dv.get_ts("BLEV")
print(spearman_factor(factor_u='BLEV', factor_j='BLEV_ME', start=start, symbol=stock_symbol))

Query data - query...
当前请求query...
{'fields': 'BLEV', 'view': 'factor'}
下载进度1760/3512.
    At fields 000748.SZ, 200011.SZ, 200012.SZ, 200016.SZ, 200017.SZ, 200018.SZ, 200019.SZ, 200020.SZ, 200022.SZ, 200025.SZ, 200026.SZ, 200028.SZ, 200029.SZ, 200030.SZ, 200037.SZ, 200045.SZ, 200053.SZ, 200054.SZ, 200055.SZ, 200056.SZ, 200058.SZ, 200152.SZ, 200160.SZ, 200168.SZ, 200413.SZ, 200418.SZ, 200429.SZ, 200468.SZ, 200488.SZ, 200505.SZ, 200512.SZ, 200521.SZ, 200530.SZ, 200539.SZ, 200541.SZ, 200550.SZ, 200553.SZ, 200570.SZ, 200581.SZ, 200596.SZ, 200613.SZ, 200625.SZ, 200706.SZ, 200725.SZ, 200726.SZ, 200761.SZ, 200771.SZ, 200869.SZ, 200986.SZ, 200992.SZ, 600005.SH, 601313.SH
Query data - daily fields prepared.
(spearmanr    0.975295
dtype: float64, array([0.]))


In [11]:
#VOL120因子复现检验
dv.add_field('VOL120', ds)
VOL120 = dv.get_ts("VOL120")
print(spearman_factor(factor_u='VOL120', factor_j='VOL121', start=start, symbol=stock_symbol))

Query data - query...
当前请求query...
{'fields': 'VOL120', 'view': 'factor'}
下载进度1760/3512.
    At fields 000748.SZ, 200011.SZ, 200012.SZ, 200016.SZ, 200017.SZ, 200018.SZ, 200019.SZ, 200020.SZ, 200022.SZ, 200025.SZ, 200026.SZ, 200028.SZ, 200029.SZ, 200030.SZ, 200037.SZ, 200045.SZ, 200053.SZ, 200054.SZ, 200055.SZ, 200056.SZ, 200058.SZ, 200152.SZ, 200160.SZ, 200168.SZ, 200413.SZ, 200418.SZ, 200429.SZ, 200468.SZ, 200488.SZ, 200505.SZ, 200512.SZ, 200521.SZ, 200530.SZ, 200539.SZ, 200541.SZ, 200550.SZ, 200553.SZ, 200570.SZ, 200581.SZ, 200596.SZ, 200613.SZ, 200625.SZ, 200706.SZ, 200725.SZ, 200726.SZ, 200761.SZ, 200771.SZ, 200869.SZ, 200986.SZ, 200992.SZ, 600005.SH, 601313.SH
Query data - daily fields prepared.
(spearmanr    0.955627
dtype: float64, array([0.]))


In [12]:
#VOL20因子复现检验
dv.add_field('VOL20', ds)
VOL120 = dv.get_ts("VOL20")
print(spearman_factor(factor_u='VOL20', factor_j='VOL201', start=start, symbol=stock_symbol))

Query data - query...
当前请求query...
{'fields': 'VOL20', 'view': 'factor'}
下载进度1760/3512.
    At fields 000748.SZ, 200011.SZ, 200012.SZ, 200016.SZ, 200017.SZ, 200018.SZ, 200019.SZ, 200020.SZ, 200022.SZ, 200025.SZ, 200026.SZ, 200028.SZ, 200029.SZ, 200030.SZ, 200037.SZ, 200045.SZ, 200053.SZ, 200054.SZ, 200055.SZ, 200056.SZ, 200058.SZ, 200152.SZ, 200160.SZ, 200168.SZ, 200413.SZ, 200418.SZ, 200429.SZ, 200468.SZ, 200488.SZ, 200505.SZ, 200512.SZ, 200521.SZ, 200530.SZ, 200539.SZ, 200541.SZ, 200550.SZ, 200553.SZ, 200570.SZ, 200581.SZ, 200596.SZ, 200613.SZ, 200625.SZ, 200706.SZ, 200725.SZ, 200726.SZ, 200761.SZ, 200771.SZ, 200869.SZ, 200986.SZ, 200992.SZ, 600005.SH, 601313.SH
Query data - daily fields prepared.
(spearmanr    0.990322
dtype: float64, array([0.]))


In [63]:
ADX_ME = sfm.ta('ADX',Close=Close,High=High,Low=Low, timeperiod=14)
dv.append_df(ADX_ME,"ADX_ME")
ADX = dv.get_ts("ADX")
ADX_ME = sfm.ta('ADX',Close=Close,High=High,Low=Low)
#########################################################
# sh300 = ds.index_daily(['000300.SH'],start,end,'close')
# sh_s = np.array(sh300[0]['close'])
# sh_data = {}
# for i in sh.columns:
#     sh_data[i] = sh_s
# sh300_r = pd.DataFrame(data=sh_data,index=sh.index).pct_change()
# data_r = dv.get_ts("close").pct_change()

Field [ADX_ME] is overwritten.
